In [10]:
import urllib.request as req
import bs4
# ssl問題，參考https://forums.raspberrypi.com/viewtopic.php?t=255167
import ssl

ctx = ssl.create_default_context()
ctx.set_ciphers('HIGH:!DH:!aNULL') 

file1 = open("ntnu_web.json", mode="w", encoding="utf-8")  # 開啟一個文件
file2 = open("ntnu_web.csv", mode="w", encoding="utf-8")  # 再開啟一個文件

In [11]:
# 定義函式抓取網頁內容
def getContent(url):
    # 讓網頁覺得我們是正常瀏覽器，所以附加Request Headers(可省略)
    request = req.Request(url, headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
    })

    with req.urlopen(request, context=ctx) as response:
        data = response.read().decode("utf-8")  # 中文編碼

    # 解析原始碼，取的每篇文章的標題
    search = bs4.BeautifulSoup(data, "html.parser")
    contents = search.find_all("div", class_="index-news-list")  # 尋找class="index-news-list"的div標籤
    
    # 定義一個list，並將每篇文章的href添加至其中
    href_list = []
    for tag in contents:
        file1.write(tag.a.text + "\n")  # 寫入資料
        file2.write(tag.a.text + "\n")  # 寫入資料
        href_list.append(tag.a["href"][:])
    return href_list

In [12]:
# 定義函式搜尋href_list所抓取href的網頁，並抓取所需資料
def main(url):
    request = req.Request(url)
    with req.urlopen(request, context=ctx) as response:
        data = response.read().decode("utf-8")  # 中文編碼

    # 解析原始碼，取的每篇文章的標題
    search = bs4.BeautifulSoup(data, "html.parser")
    news = search.find_all("div", class_="news-content")  # 尋找class="news-content"的div標籤
    
    for tag in news:
        lis1 = tag.find_all("h3") # 抓取所有字號3的標題
        for h3 in lis1:
            file1.write(h3.text + "\n")  # 寫入資料
            file2.write(h3.text + "\n")  # 寫入資料
        lis2 = tag.find_all("p") # 抓取所有<p>標籤內的文字
        for p in lis2:
            file1.write(p.text + "\n")  # 寫入資料
            file2.write(p.text + "\n")  # 寫入資料
        lis3 = tag.find_all("li") # 抓取所有<li>標籤內的文字
        for li in lis3:
            file1.write(li.text + "\n")  # 寫入資料
            file2.write(li.text + "\n")  # 寫入資料
        lis4 = tag.find_all("div") # 抓取所有<div>標籤內的文字
        for div in lis4:
            if div.text != "回上一頁": # 不寫入"回上一頁"
                file1.write(div.text + "\n")  # 寫入資料
                file2.write(div.text + "\n")  # 寫入資料

In [13]:
# 主程式
pageURL = "https://www.ntnu.edu.tw/"
file1.write("--------第1頁--------" + "\n")
file2.write("--------第1頁--------" + "\n")
href_list = getContent(pageURL)

# 利用迴圈重複搜尋網頁
count = 1
# 校網僅有20個在class="news-content"的div標籤內的文章，加上首頁共21頁
# 數字設定超過21會出，程式可執行但21頁後無資料可抓取
while count < 21:
    # 首個數值在href_list[0]
    nextURL = href_list[count-1]
    # 寫入從第2頁
    file1.write("--------第" + str(count+1) + "頁--------" + "\n")
    file2.write("--------第" + str(count+1) + "頁--------" + "\n")
    main(nextURL)
    count += 1

file1.close()  # 關閉文件
file2.close()  # 關閉文件